In [12]:
import cv2
import mediapipe as mp
import math
import numpy as np

In [13]:
video = cv2.VideoCapture(filename="videos/rope_pushdown.mp4")

mpPose = mp.solutions.pose
pose = mpPose.Pose()

drawing = mp.solutions.drawing_utils

dir, count = 0, 0

def findAngle(img, p1, p2, p3, landmarkList, draw=True):
    x1, y1 = landmarkList[p1][1:]
    x2, y2 = landmarkList[p2][1:]
    x3, y3 = landmarkList[p3][1:]
    
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    if angle <= 0: angle += 360
    
    cv2.line(img=img, pt1=(x1, y1), pt2=(x2, y2), color=(0, 255, 255), thickness=3)
    cv2.line(img=img, pt1=(x2, y2), pt2=(x3, y3), color=(0, 255, 255), thickness=3)
    
    cv2.circle(img=img, center=(x1, y1), radius=10, color=(0, 255, 255), thickness=cv2.FILLED)
    cv2.circle(img=img, center=(x2, y2), radius=10, color=(0, 255, 255), thickness=cv2.FILLED)
    cv2.circle(img=img, center=(x3, y3), radius=10, color=(0, 255, 255), thickness=cv2.FILLED)
    
    cv2.circle(img=img, center=(x1, y1), radius=14, color=(0, 255, 255), thickness=3)
    cv2.circle(img=img, center=(x2, y2), radius=14, color=(0, 255, 255), thickness=3)
    cv2.circle(img=img, center=(x3, y3), radius=14, color=(0, 255, 255), thickness=3)
    
    cv2.putText(img=img,
                text=f"{int(angle)}",
                org=(x2-20, y2-10),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1,
                color=(0, 255, 255),
                thickness=3)
    return angle

while True:
    retval, img = video.read()
    if not retval: break
    img = cv2.resize(src=img, dsize=(398, 698))
    
    
    landmarkList = []
    results = pose.process(image=img).pose_landmarks
    if results:
        drawing.draw_landmarks(image=img,
                               landmark_list=results,
                               connections=mpPose.POSE_CONNECTIONS)
    
        for id, landmark in enumerate(results.landmark):
            height, width, color = img.shape
            cx, cy = int(landmark.x * width), int(landmark.y * height)
            landmarkList.append([id, cx, cy])
            
        angle = findAngle(img=img, p1=12, p2=14, p3=16, landmarkList=landmarkList, draw=True)
        per = np.interp(x=angle, xp=(80, 90), fp=(0, 100))
        
        if per == 100:
            if dir == 0:
                dir = 1
                count += 0.5
        
        if per == 0:
            if dir == 1:
                dir = 0
                count += 0.5
                
        
                
        cv2.putText(img=img,
                    text=f"Tekrar: {int(count)}",
                    org=(30, 40),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1,
                    color=(0, 255, 255),
                    thickness=3)
    
    cv2.imshow(winname="Video", mat=img)
    if cv2.waitKey(10) == ord("q"): break
    
video.release()
cv2.destroyAllWindows()